# **Simulation in closed-loop**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

import package_LAB
from importlib import reload
package_LAB = reload(package_LAB)

from package_LAB import LL_RT, PID_RT
from package_DBR import SelectPath_RT, Delay_RT


In [2]:
#plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interactive, VBox

## Simulation parameters

In [3]:
TSim = 1000
Ts = 0.5    
N = int(TSim/Ts) + 1 

## Dynamics and controller parameters

In [4]:
#DV 
Kp_ODV_SOPDT = 0.2951290424136788
T1_ODV_SOPDT = 182.2549613489765
T2_ODV_SOPDT = 13.184430234847984
theta_ODV_SOPDT = 28.999891911961512

#MV
Kp_OMV_SOPDT = 0.30788564834253684
T1_OMV_SOPDT = 183.81942938046797
T2_OMV_SOPDT = 3.2920224028341535e-12
theta_OMV_SOPDT = 20.015407110302775

#Operating points 
DV0 = 50 
MV0 = 50
PV0 = 49.3

# Set maximum and minimum MV values
MVmin = 0
MVmax = 100

## Scenarios

In [5]:
# Scenarios :
# Path for SP, MAN, MV_MAN, FF

scenario = 6 #1 to 6

if scenario == 1:
    # scenario 1: Change in set point
    ManPath = {0: True, 180: False}
    MVmanPath = {0: 50}
    SPPath = {0: PV0, 250: PV0+10}
    FFPath = {0: False}
    DVPATh = {0:50}
elif scenario == 2:
    # scenario 2: Manual mode no FF, response to perturbation
    ManPath = {0: True}
    MVmanPath = {0: 50}
    SPPath = {0: PV0}
    FFPath = {0: False}
    DVPATh = {0:50, 500: 70}
elif scenario == 3:
    # scenario 3: Manual mode with FF, response to perturbation
    ManPath = {0: True}
    MVmanPath = {0: 50}
    SPPath = {0: PV0}
    FFPath = {0: True}
    DVPATh = {0:50, 400: 70}
elif scenario == 4:
    # scenario 4: Automatic mode no FF, response to perturbation
    ManPath = {0: False}
    MVmanPath = {0: 50}
    SPPath = {0: PV0}
    FFPath = {0: False}
    DVPATh = {0:50, 300: 70}
elif scenario == 5:
    # scenario 5: Automatic mode with FF, response to perturbation
    ManPath = {0: False}
    MVmanPath = {0: 50}
    SPPath = {0: PV0}
    FFPath = {0: True}
    DVPATh = {0:50, 300: 70}
elif scenario == 6:
    # scenario 6: Automatic mode with FF, response to set point change and perturbation
    SPPath = {0:PV0, 400: PV0+10}
    ManPath = {0: False}
    MVmanPath = {0: 0}
    FFPath = {0: True}
    DVPATh = {0:50, 650:30}

## Running simulation

In [6]:
#Initialize arrays of data
t = []

SP = []
PV = []
MAN = []
MV_MAN = []
FF = []
PV_P = []
DV = []
PV_D = []
MV_FF = []

MV = []
MVp = []
MVi = []
MVd = []
E = []

#intermediate MV values
MV_P_delay = []
MV_P_FO_delay = []
MVFF_delay = []
MVFF_LL = []
MV_D_delay = []
MV_D_FO_delay = []

for i in range(0,N):
    t.append(i*Ts)
    SelectPath_RT(SPPath,t,SP)
    SelectPath_RT(ManPath,t,MAN)
    SelectPath_RT(MVmanPath,t,MV_MAN) 
    SelectPath_RT(FFPath,t,FF)
    SelectPath_RT(DVPATh,t,DV)
    
    #Feed Forward
    Delay_RT(DV-DV0*np.ones_like(DV), np.max([theta_ODV_SOPDT-theta_OMV_SOPDT,0]), Ts, MVFF_delay)
    LL_RT(MVFF_delay,-Kd/Kp,T1p,T1d,Ts,MVFF_LL)
    LL_RT(MVFF_LL,1,T2p,T2d,Ts,MV_FF)
    
    #PID
    PID_RT(SP, PV, MAN, MV_MAN, FF, MV_FF, PVInit, Kc, Ti, Td, alpha, MVmin, MVmax, Ts, MV, MVp, MVi, MVd, E)
    
    #Process
    Delay_RT(MV,thetap,Ts, MV_P_delay, MV0) 
    FO_RT(MV_P_delay, Kp, T1p, Ts, MV_P_FO_delay, 0)
    FO_RT(MV_P_FO_delay, 1, T2p, Ts, PV_P, 0)
        
    #Disturbance
    Delay_RT(DV-DV0*np.ones_like(DV), thetad, Ts, MV_D_delay, 0)
    FO_RT(MV_D_delay, Kd, T1d, Ts, MV_D_FO_delay, 0)
    FO_RT(MV_D_FO_delay, 1, T2d, Ts,PV_D, 0)
    
    PV.append(PV_P[-1] + PV_D[-1] + PV0 - Kp * MV0)